## Failed attempt at measuring throughput using the process-function 

In [2]:
# Load dependencies and set constants
from pyflink.datastream.functions import ProcessFunction
from pyflink.common.serialization import SimpleStringSchema, SerializationSchema
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment, TimeCharacteristic
from pyflink.datastream.connectors import FlinkKafkaConsumer, FlinkKafkaProducer
from matplotlib import pyplot as plt

import time
import sys

producer_props = {
    'bootstrap.servers': 'localhost:9092',
    'queue.buffering.max.messages': '1000000'
}

consumer_props = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "twitter-consumers",
    "client.id": "client-1",
}

KAFKA_TOPIC = "twitter-stream"
KAFKA_CONNECTOR_JAR = "file:///home/ubuntu/flink-sql-connector-kafka_2.11-1.12.2.jar"

# Number of tweets in the dataset (10 MB)
NUMBER_OF_TWEETS = 14484

# Number of times the dataset is produced by Kafka
NUMBER_OF_PRODUCTIONS = 100

DATASET_SIZE = 10

NUMBER_OF_EXECUTIONS = 10

measured_times = []
records_received = 0
start_time = -1
iterations = 0
kafka_props = {'bootstrap.servers': 'localhost:9092', 'group.id': 'twitter_consumers'}


def collect_stats():
    throughput_mbs = [(DATASET_SIZE * NUMBER_OF_PRODUCTIONS) / x for x in measured_times]
    throughput_messages = [NUMBER_OF_TWEETS / x for x in measured_times]
    min_throughput = min(throughput_mbs)
    max_throughput = max(throughput_mbs)
    avg_throughput = mean(throughput_mbs)

    print("============ Throughput Results ============")
    for i in len(throughput_mbs):
        print("Iteration {]: {} MB/s | {} Msgs/s".format(str(i), throughput_mbs[i], throughput_messages[i]))

    # Plot Max, Min, Avg MB/s
    fig = plt.figure(figsize=(10, 6), dpi=80)
    plt.bar(0, max_throughput, width=1, color='navy')
    plt.bar(1, avg_throughput, width=1, color='darkcyan')
    plt.bar(2, min_throughput, width=1, color='skyblue')

    plt.tick_params(labelleft=True, labelbottom=False)
    plt.legend(["Maximum MB/s", "Average MB/s", "Minimum MB/s"], prop={'size': 12}, bbox_to_anchor=(1.05, 0.8))
    plt.title("Summarization of Throughput in MB/s", fontsize=14, pad=12)
    plt.savefig('throughput.png')


class MyProcessFunction(ProcessFunction):

    def process_element(self, value, ctx: 'ProcessFunction.Context'):
        global records_received
        global start_time
        global iterations
        cur_time = time.time()
        latency = (cur_time * 1000) - ctx.timestamp()
        result = str(latency)
        #yield result
        if records_received == 0:
            start_time = cur_time
        records_received += 1
        if records_received % (NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS) == 0:
            end_time = cur_time()
            total_time = end_time - start_time
            measured_times.append(total_time)
            records_received = 0
            iterations += 1
            if iterations == 10:
                collect_stats()
        yield result


env = StreamExecutionEnvironment.get_execution_environment()
env.set_stream_time_characteristic(TimeCharacteristic.EventTime)


env.add_jars(KAFKA_CONNECTOR_JAR)
env.add_classpaths(KAFKA_CONNECTOR_JAR)

kafka_consumer = FlinkKafkaConsumer("twitter-stream", SimpleStringSchema(), consumer_props)
kafka_producer = FlinkKafkaProducer("twitter-stream-results", SimpleStringSchema(), producer_props)

stream = env.add_source(kafka_consumer)
stream.process(MyProcessFunction(), output_type=Types.STRING()) \
      .add_sink(kafka_producer)
env.execute()






KeyboardInterrupt: 